In [24]:
import numpy as np
import pandas as pd
import geopandas as gp
import os
import subprocess
import math
import shapely as shp
from datetime import datetime
from fiona.crs import from_epsg
import matplotlib.pylab as plt
import re
from pandas.tseries.holiday import get_calendar, HolidayCalendarFactory, GoodFriday
from datetime import datetime
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from itertools import compress

np.random.seed(222)

%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
wd = os.getcwd()
os.chdir('../data')
os.getcwd()

'/Users/geoffperrin/Desktop/DLBA/demo_project/data'

In [3]:
#read in detroit feature dataset
city_df = pd.read_csv('CitywideIndicators_20170123.csv')

/Users/geoffperrin/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#only keep DLBA Owned properties for the training data
dlba_df = city_df[city_df['Property Ownership'] == 'DLBA Owned']

#remove lots
dlba_df = dlba_df[dlba_df['Property Class'] != 'Residential-Lot']

dlba_df_small = dlba_df.rename(columns=lambda x: x.replace(' ', '_'))
dlba_df_small.head()

,Account_ID,Parcel_ID,Address,Property_Ownership,Property_Class,Program,Current_Status,Current_Status_Date,Neighborhood,Neighborhood_Sales_Median,...,MCM_Institutional_Use,MCM_Public_Use,MCM_Condition,MCM_Fire_Damage,MCM_Level_of_Fire_Damage,MCM_Boarding_Needed,MCM_Trespass,MCM_Vacant_Lot,MCM_Maintained,MCM_Dumping
3,001j000000h4Lo4,21021624.,16272 Lappin,DLBA Owned,Unknown-Lot,Inventory,Intake Review,1/29/2016,Regent Park,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,001j000000Idu4s,21007166.,13064 Elmdale,DLBA Owned,Residential-Structure,Own It Now,Sales Review,5/26/2016,Outer Drive-Hayes,1.40,...,NaN,NaN,Fair,No,NaN,No,NaN,NaN,NaN,Yes
62,001j000000Idu4y,21007448.001,14291 Elmdale,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,4/1/2016,Outer Drive-Hayes,1.40,...,NaN,NaN,Suggest Demolition,Yes,Major,Yes,Open,NaN,NaN,Yes
104,001j000000Idu5O,19005023.,4180 Cadillac,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,3/31/2016,Gratiot Woods,0.59,...,NaN,NaN,Poor,No,NaN,Yes,NaN,NaN,NaN,No
116,001j000000Idu5U,19005022.,4174 Cadillac,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,3/31/2016,Gratiot Woods,0.59,...,NaN,NaN,Fair,No,NaN,Yes,NaN,NaN,NaN,No


## need to fix nans in neighborhood sales median
### all the neighborhoods should be in there, but some rows have NaNs!

In [5]:
dlba_df_small[pd.isnull(dlba_df_small.Neighborhood_Sales_Median)]

,Account_ID,Parcel_ID,Address,Property_Ownership,Property_Class,Program,Current_Status,Current_Status_Date,Neighborhood,Neighborhood_Sales_Median,...,MCM_Institutional_Use,MCM_Public_Use,MCM_Condition,MCM_Fire_Damage,MCM_Level_of_Fire_Damage,MCM_Boarding_Needed,MCM_Trespass,MCM_Vacant_Lot,MCM_Maintained,MCM_Dumping
3,001j000000h4Lo4,21021624.,16272 Lappin,DLBA Owned,Unknown-Lot,Inventory,Intake Review,1/29/2016,Regent Park,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17148,001j000000IdyLO,21077368.,5724 University Pl,DLBA Owned,Unknown-Lot,Inventory,Intake Review,1/15/2016,Cornerstone Village,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Improved,Yes,No
28850,001j000000Ie1nC,16000117.,5671 Harvey,DLBA Owned,Residential-Structure,Inventory,Intake Review,3/18/2015,Delray,NaN,...,NaN,NaN,Suggest Demolition,Yes,NaN,Yes,NaN,NaN,NaN,Yes
31065,001j000000Ie2Aa,12009680-6,769 25th St,DLBA Owned,NonResidential-Lot,Inventory,Intake Review,6/16/2015,West Side Industrial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Improved,Yes,No
57416,001j000000Ie9AQ,06001207.003L,1430 W Hancock,DLBA Owned,NonResidential-Lot,Inventory,Intake Review,12/8/2016,Wayne State,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unimproved,Yes,No
73481,001j000000IeBVG,21062227.,4847 Ashland,DLBA Owned,Residential-Structure,Inventory,Intake Review,1/15/2016,Fox Creek,NaN,...,NaN,NaN,Good,No,NaN,No,NaN,NaN,NaN,No
77753,001j000000Iechp,13012559.,19405 Gable,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,5/2/2016,Farwell,NaN,...,NaN,NaN,Suggest Demolition,Yes,Collapsed,Yes,Open,NaN,NaN,No
81036,001j000000IecUX,22119293-4,17424 Marene,DLBA Owned,Residential-Structure,Inventory,Intake Review,1/19/2017,Oak Grove,NaN,...,NaN,NaN,Fair,No,NaN,No,NaN,NaN,NaN,No
82705,001j000000IeD4p,21000400.,12440 E Jefferson,DLBA Owned,NonResidential-Lot,Inventory,Intake Review,1/15/2016,Conner Creek Industrial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Improved,Yes,No
82747,001j000000IeD5f,21045809-27,1026 Tennessee,DLBA Owned,NonResidential-Lot,Inventory,Intake Review,6/16/2015,Conner Creek Industrial,NaN,...,NaN,Other,NaN,NaN,NaN,NaN,NaN,Improved,Yes,No


In [6]:
dlba_df_neighborhood = dlba_df_small[['Neighborhood', 'Neighborhood_Sales_Median']]
dlba_df_neighborhood.groupby(['Neighborhood'], as_index=False).mean()

,Neighborhood,Neighborhood_Sales_Median
0,Airport Sub,0.570000
1,Aviation Sub,6.240000
2,Bagley,11.680000
3,Barton-McFarland,1.240000
4,Belmont,3.470000
5,Bentler-Pickford,1.670000
6,Berg-Lahser,44.434000
7,Blackstone Park,7.940000
8,Boston Edison,12.890000
9,Boynton,1.580000


In [7]:
#read in detroit parcel distance to school df
school_df = pd.read_csv('DistanceToSchool_20170123.csv')
school_df_small = school_df[["PARCELNO", "DistanceToSchool_FT"]]
school_df_small.columns = ['Parcel_ID', 'dist_to_school']
school_df_small.head()

,Parcel_ID,dist_to_school
0,16006256.,2196.731029
1,04000103-4,2375.448135
2,04003341.,2442.534185
3,04003339-40,2510.715691
4,04000119.,2341.253487


In [8]:
#merge distance to school to city feature df
dlba_school_df = dlba_df_small.merge(school_df_small, on='Parcel_ID')
dlba_school_df.head()

,Account_ID,Parcel_ID,Address,Property_Ownership,Property_Class,Program,Current_Status,Current_Status_Date,Neighborhood,Neighborhood_Sales_Median,...,MCM_Public_Use,MCM_Condition,MCM_Fire_Damage,MCM_Level_of_Fire_Damage,MCM_Boarding_Needed,MCM_Trespass,MCM_Vacant_Lot,MCM_Maintained,MCM_Dumping,dist_to_school
0,001j000000Idu4s,21007166.,13064 Elmdale,DLBA Owned,Residential-Structure,Own It Now,Sales Review,5/26/2016,Outer Drive-Hayes,1.40,...,NaN,Fair,No,NaN,No,NaN,NaN,NaN,Yes,1235.838510
1,001j000000Idu4y,21007448.001,14291 Elmdale,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,4/1/2016,Outer Drive-Hayes,1.40,...,NaN,Suggest Demolition,Yes,Major,Yes,Open,NaN,NaN,Yes,2823.745964
2,001j000000Idu5O,19005023.,4180 Cadillac,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,3/31/2016,Gratiot Woods,0.59,...,NaN,Poor,No,NaN,Yes,NaN,NaN,NaN,No,758.959715
3,001j000000Idu5U,19005022.,4174 Cadillac,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,3/31/2016,Gratiot Woods,0.59,...,NaN,Fair,No,NaN,Yes,NaN,NaN,NaN,No,788.673532
4,001j000000Idu5y,18013649.,7729 American,DLBA Owned,Residential-Structure,Inventory,Intake Review,3/18/2015,Midwest,0.84,...,NaN,Poor,No,NaN,Yes,NaN,NaN,NaN,No,4959.561810


### Notes: current status is informative, but program is also informative

### perhaps add a feature that is the number of occupied properties within a buffer of every property - alice will send over parcel_ID lat/long data

## Notes 2-1-17
1. add MCM condition to dummy variables
2. add any houses that score very high on the occupancy model as homes that should not be demolished
3. update DFD fire list
4. figure out regularization techniques for logit "regression" model

In [9]:
len(dlba_school_df)

34014

In [10]:
#code up demo decision yes / no
demo_decision_dict = {'Demo Awaiting Funding': 1,
        'Demo Pipeline': 1,
        'Demo Contracted': 1,
        'Demo Review': 1,
        'Demolished': 999,
        'Construction Complete': 0,
        'Occupied': 0,
        'Sold': 0,
        'Institutional Partner Coordination': 999,
        'For Sale': 0,
        'Sales Review': 0,
        'Sales Pipeline': 0,
        'Vacant Land': 999,
        'Sales Hold': 0,
        'Under Construction': 999,
        'Lease/ROE/TLA': 999,
        'Warehouse': 999,
        'NPOC Isolation - Do not move!': 999,
        'NSP Quarantine': 999,
        'Reconveyance Complete': 999,
        'Title Problem Identified': 999,
        'DJ Quarantine': 999,
        'PDD Quarantine': 999,
        'Intake Review': 999}

dlba_school_df['demo_decision'] = 0
dlba_school_df['demo_decision'] = [demo_decision_dict[i] for i in dlba_school_df["Current_Status"]]
dlba_school_df = dlba_school_df[dlba_school_df['demo_decision'] != 999]

dlba_school_df.head()

,Account_ID,Parcel_ID,Address,Property_Ownership,Property_Class,Program,Current_Status,Current_Status_Date,Neighborhood,Neighborhood_Sales_Median,...,MCM_Condition,MCM_Fire_Damage,MCM_Level_of_Fire_Damage,MCM_Boarding_Needed,MCM_Trespass,MCM_Vacant_Lot,MCM_Maintained,MCM_Dumping,dist_to_school,demo_decision
0,001j000000Idu4s,21007166.,13064 Elmdale,DLBA Owned,Residential-Structure,Own It Now,Sales Review,5/26/2016,Outer Drive-Hayes,1.40,...,Fair,No,NaN,No,NaN,NaN,NaN,Yes,1235.838510,0
1,001j000000Idu4y,21007448.001,14291 Elmdale,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,4/1/2016,Outer Drive-Hayes,1.40,...,Suggest Demolition,Yes,Major,Yes,Open,NaN,NaN,Yes,2823.745964,1
2,001j000000Idu5O,19005023.,4180 Cadillac,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,3/31/2016,Gratiot Woods,0.59,...,Poor,No,NaN,Yes,NaN,NaN,NaN,No,758.959715,1
3,001j000000Idu5U,19005022.,4174 Cadillac,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,3/31/2016,Gratiot Woods,0.59,...,Fair,No,NaN,Yes,NaN,NaN,NaN,No,788.673532,1
8,001j000000Idu66,18013642.,7777 American,DLBA Owned,Residential-Structure,Non-HHF Demo,Demo Awaiting Funding,3/28/2016,Midwest,0.84,...,Poor,No,NaN,Yes,NaN,NaN,NaN,No,5197.653491,1


In [11]:
len(dlba_school_df)

19579

In [12]:
dlba_school_df[~pd.isnull(dlba_school_df['Neighborhood_Sales_Median'])]

,Account_ID,Parcel_ID,Address,Property_Ownership,Property_Class,Program,Current_Status,Current_Status_Date,Neighborhood,Neighborhood_Sales_Median,...,MCM_Condition,MCM_Fire_Damage,MCM_Level_of_Fire_Damage,MCM_Boarding_Needed,MCM_Trespass,MCM_Vacant_Lot,MCM_Maintained,MCM_Dumping,dist_to_school,demo_decision
0,001j000000Idu4s,21007166.,13064 Elmdale,DLBA Owned,Residential-Structure,Own It Now,Sales Review,5/26/2016,Outer Drive-Hayes,1.40,...,Fair,No,NaN,No,NaN,NaN,NaN,Yes,1235.838510,0
1,001j000000Idu4y,21007448.001,14291 Elmdale,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,4/1/2016,Outer Drive-Hayes,1.40,...,Suggest Demolition,Yes,Major,Yes,Open,NaN,NaN,Yes,2823.745964,1
2,001j000000Idu5O,19005023.,4180 Cadillac,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,3/31/2016,Gratiot Woods,0.59,...,Poor,No,NaN,Yes,NaN,NaN,NaN,No,758.959715,1
3,001j000000Idu5U,19005022.,4174 Cadillac,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,3/31/2016,Gratiot Woods,0.59,...,Fair,No,NaN,Yes,NaN,NaN,NaN,No,788.673532,1
8,001j000000Idu66,18013642.,7777 American,DLBA Owned,Residential-Structure,Non-HHF Demo,Demo Awaiting Funding,3/28/2016,Midwest,0.84,...,Poor,No,NaN,Yes,NaN,NaN,NaN,No,5197.653491,1
9,001j000000Idu67,19009910.,6432 Crane,DLBA Owned,Residential-Structure,Own It Now,Sales Review,5/26/2016,Airport Sub,0.57,...,Good,No,NaN,No,NaN,NaN,NaN,No,1733.394976,0
12,001j000000Idu6R,18013640.,7791 American,DLBA Owned,Residential-Structure,Non-HHF Demo,Demo Pipeline,10/6/2016,Midwest,0.84,...,Poor,No,NaN,No,NaN,NaN,NaN,No,5265.587593,1
14,001j000000Idu6y,12009331.,4822 25th St,DLBA Owned,Residential-Structure,Buy Back,Sales Pipeline,5/13/2016,Core City,0.34,...,Good,No,NaN,No,NaN,NaN,NaN,No,688.020669,0
15,001j000000Idu7A,18013690.,7273 American,DLBA Owned,Residential-Structure,Non-HHF Demo,Demo Awaiting Funding,3/28/2016,Midwest,0.84,...,Poor,No,NaN,Yes,NaN,NaN,NaN,No,3469.686595,1
17,001j000000Idu88,22076090.,6516 Rosemont,DLBA Owned,Residential-Structure,Auction,For Sale,12/7/2016,Warrendale,2.77,...,Fair,No,NaN,Yes,Secure,NaN,NaN,No,1993.681730,0


In [13]:
#code dummy variables
dlba_school_df['dng_dummy'] = 0
dlba_school_df['dng_dummy'] = [0 if pd.isnull(i) else 1 for i in dlba_school_df['DNG_Number']]

dlba_school_df['fire_dummy'] = 0
dlba_school_df['fire_dummy'] = [0 if pd.isnull(i) else 1 for i in dlba_school_df['DFD_Fire:_Occurrence_Date']]

dlba_school_df['usps_vacant_dummy'] = 0
dlba_school_df['usps_vacant_dummy'] = [0 if pd.isnull(i) else 1 for i in dlba_school_df['USPS:_Vacancy_Indicated']]

dlba_school_df['foreclosed_dummy'] = 0
dlba_school_df['foreclosed_dummy'] = [0 if pd.isnull(i) else 1 for i in dlba_school_df['Foreclosure_Year']]

dlba_school_df['foreclosure_lag'] = 0
dlba_school_df['foreclosure_lag'] = [0 if pd.isnull(i) else (2017 - i) for i in dlba_school_df['Foreclosure_Year']]

dlba_school_df['boarding_needed_dummy'] = 0
dlba_school_df['boarding_needed_dummy'] = [0 if (pd.isnull(i) or i == 'No') else 1 for i in dlba_school_df['MCM_Boarding_Needed']]

dlba_school_df['dumping_dummy'] = 0
dlba_school_df['dumping_dummy'] = [0 if (pd.isnull(i) or i == 'No') else 1 for i in dlba_school_df['MCM_Dumping']]

#replace NaNs with zeros
dlba_school_df['Occupancy_Model:_Probability_Pct'] = [0 if pd.isnull(i) else i for i in dlba_school_df['Occupancy_Model:_Probability_Pct']]

#remove NaNs for Neighborhood_Sales_Median
#in final roll out I need to improve this!!!
dlba_school_df = dlba_school_df[~pd.isnull(dlba_school_df['Neighborhood_Sales_Median'])]

dlba_school_df_small = dlba_school_df.drop(['Account_ID', 'Property_Ownership', 'Property_Class', 'Program', \
                                            'Current_Status', 'Current_Status_Date', 'Neighborhood', 'DNG_Number', \
                                            'WCT_Foreclosure_Status', \
                                            'MCM_Blext_Created_At', 'MCM_Structure', 'MCM_Occupancy',  'MCM_Use',\
                                           'MCM_Units', 'MCM_Residence_Type', 'MCM_Commercial_Use', 'MCM_Industrial_Use',\
                                           'MCM_Institutional_Use', 'MCM_Public_Use', 'MCM_Level_of_Fire_Damage', \
                                            'MCM_Trespass', 'MCM_Vacant_Lot', 'MCM_Maintained', 'DFD_Fire:_Occurrence_Date',\
                                           'USPS:_Vacancy_Indicated', 'Foreclosure_Year', 'MCM_Fire_Damage',\
                                           'USPS:_Last_Update', 'Occupancy_Model:_Last_Update', 'MCM_Boarding_Needed',\
                                           'MCM_Dumping', 'MCM_Condition'], 1)
dlba_school_df_small.head(5)

,Parcel_ID,Address,Neighborhood_Sales_Median,Occupancy_Model:_Probability_Pct,dist_to_school,demo_decision,fire_dummy,usps_vacant_dummy,foreclosed_dummy,foreclosure_lag,boarding_needed_dummy,dumping_dummy
0,21007166.,13064 Elmdale,1.40,14.54,1235.838510,0,0,1,1,4.0,0,1
1,21007448.001,14291 Elmdale,1.40,14.54,2823.745964,1,0,1,0,0.0,1,1
2,19005023.,4180 Cadillac,0.59,14.54,758.959715,1,0,1,1,6.0,1,0
3,19005022.,4174 Cadillac,0.59,14.54,788.673532,1,0,1,0,0.0,1,0
8,18013642.,7777 American,0.84,14.54,5197.653491,1,0,1,1,4.0,1,0


In [14]:
dlba_school_df_small = dlba_school_df_small.rename(columns={'Occupancy_Model:_Probability_Pct': 'occupancy_model_pct'})
dlba_school_df_small = dlba_school_df_small[['Parcel_ID', 'Address', 'demo_decision', 'Neighborhood_Sales_Median',\
                                            'occupancy_model_pct', 'dist_to_school',\
                                            'fire_dummy', 'usps_vacant_dummy', 'foreclosed_dummy',\
                                            'foreclosure_lag', 'boarding_needed_dummy', 'dumping_dummy', 'dng_dummy']]

#fit model 
OS = []
IS = []

y = dlba_school_df_small.demo_decision
X = dlba_school_df_small.iloc[:,3:]
'''
let's try this without the crossvalidation loop first


for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = i)
    
    #learn logistic regression over the training sample
    logit = LogisticRegression(C=1)
    logit.fit(X_train, y_train)
    # C here controls the regularization strength--which will be discussed in Bayesian session
    # For now just keep in mind that setting a high value will make the regularization ineffective

    #apply to the validation sample
    labelPred = logit.predict(X_test)
    #labelPred = logPred > 0.5
    labelActual = y_test
    
'''    


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)
    
#learn logistic regression over the training sample
logit = LogisticRegression(C=1)
logit.fit(X_train, y_train)
# C here controls the regularization strength--which will be discussed in Bayesian session
# For now just keep in mind that setting a high value will make the regularization ineffective

#apply to the validation sample
labelPred = logit.predict(X_test)
#labelPred = logPred > 0.5
labelActual = y_test
    
logitProbability = logit.predict_proba(X_test)
(logitProbability)
    
    

array([[ 0.95912128,  0.04087872],
       [ 0.52941819,  0.47058181],
       [ 0.1993429 ,  0.8006571 ],
       ..., 
       [ 0.53672128,  0.46327872],
       [ 0.83056074,  0.16943926],
       [ 0.51008672,  0.48991328]])

In [15]:
logit.coef_

array([[ -9.32504884e-02,  -3.04027908e-02,   1.21251915e-04,
          8.87576988e-01,  -4.44969491e-02,  -3.82111845e-01,
          1.54594746e-01,   1.04198437e+00,   3.24754588e-01]])

In [17]:
logit.get_params()

{'C': 1,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [20]:
sklearn.feature_selection.chi2(X,y)

(array([  4.78581548e+02,   4.53133609e+04,   1.02913438e+05,
          2.60806735e+02,   1.38095545e+00,   6.97457752e+01,
          1.13661758e+03,   1.18880157e+03,   3.68514499e+02]),
 array([  4.34907274e-106,   0.00000000e+000,   0.00000000e+000,
          1.14463819e-058,   2.39938466e-001,   6.74621972e-017,
          3.63396861e-249,   1.65591995e-260,   3.94126261e-082]))

In [31]:
r_tree = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
r_tree.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [32]:
r_tree.feature_importances_

array([ 0.25503989,  0.11422384,  0.42771391,  0.01503209,  0.00963992,
        0.00594898,  0.07525826,  0.07304165,  0.02410146])

In [36]:
X_train.head()

,Neighborhood_Sales_Median,occupancy_model_pct,dist_to_school,fire_dummy,usps_vacant_dummy,foreclosed_dummy,foreclosure_lag,boarding_needed_dummy,dumping_dummy
27763,1.80,14.54,873.279624,0,1,1,3.0,1,1
21357,2.53,14.54,3252.708703,0,1,1,4.0,0,0
20093,1.61,14.54,728.920730,0,1,0,0.0,0,0
33922,6.55,14.54,3196.620815,0,1,1,4.0,0,0
29996,2.73,14.54,2277.113852,0,1,0,0.0,0,1


In [34]:
dlba_school_df_small.head(50)

,Parcel_ID,Address,demo_decision,Neighborhood_Sales_Median,occupancy_model_pct,dist_to_school,fire_dummy,usps_vacant_dummy,foreclosed_dummy,foreclosure_lag,boarding_needed_dummy,dumping_dummy
0,21007166.,13064 Elmdale,0,1.40,14.54,1235.838510,0,1,1,4.0,0,1
1,21007448.001,14291 Elmdale,1,1.40,14.54,2823.745964,0,1,0,0.0,1,1
2,19005023.,4180 Cadillac,1,0.59,14.54,758.959715,0,1,1,6.0,1,0
3,19005022.,4174 Cadillac,1,0.59,14.54,788.673532,0,1,0,0.0,1,0
8,18013642.,7777 American,1,0.84,14.54,5197.653491,0,1,1,4.0,1,0
9,19009910.,6432 Crane,0,0.57,14.54,1733.394976,0,1,0,0.0,0,0
12,18013640.,7791 American,1,0.84,14.54,5265.587593,0,1,1,6.0,0,0
14,12009331.,4822 25th St,0,0.34,100.00,688.020669,0,1,1,4.0,0,0
15,18013690.,7273 American,1,0.84,58.72,3469.686595,0,1,1,5.0,1,0
17,22076090.,6516 Rosemont,0,2.77,14.54,1993.681730,0,1,0,0.0,1,0
